<a href="https://colab.research.google.com/github/GurjeetSinghSangra/MachineLearningAssignment/blob/main/emnist_DBN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
##################################################
# Imports
##################################################

import numpy as np
import scipy.io
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn #torch made by layers, in nn we have linear layer which implements standard forward fully connected layer
import torch.nn.functional as F 
import torchvision #used for data loading
import torch.optim as optim #optimizer for the gradient
from tqdm.notebook import tqdm
import time


# Setting the device
if torch.cuda.is_available(): 
    print('GPU enabled!')
    device = torch.device("cuda:0")
else:
    print('You are not using the GPU, activate it following:')
    

You are not using the GPU, activate it following:


In [ ]:
def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image
    
images = np.apply_along_axis(rotate, 1, x_train)
plt.imshow(images[1,], cmap=plt.get_cmap('gray')) 

In [218]:
class DBN(nn.Module):
  DN = {}
  DN['layersize'] = np.array([50, 50, 200])
  DN['maxepochs'] = 10
  DN['batchsize'] = 500

  def __init__(self, layers_nn, maxepochs=20, batchsize=500, epsilonw=0.1, \
                               sparsity=1, spars_factor=0.04, epsilonvb=0.1, epsilonhb=0.1, weightcost=0.0002, init_momentum=0.5, final_momentum=0.9):
    super(DBN, self).__init__()
    self.DN['layersize'] = layers_nn
    self.DN['nlayers'] = len(layers_nn)
    self.DN['batchsize'] = batchsize
    self.DN['maxepochs'] = maxepochs
    self.epsilonw_GPU = epsilonw
    self.epsilonvb=epsilonvb
    self.epsilonhb=epsilonhb
    self.weightcost = weightcost
    self.initialmomentum = init_momentum
    self.finalmomentum = final_momentum
    self.sparsity = sparsity
    self.spars_factor = spars_factor
    if self.DN['maxepochs']  > 5:
      self.momentum = self.finalmomentum
    else:
      self.momentum = self.initialmomentum

  def load_data(self, fname):
    dataset = scipy.io.loadmat(fname)
    data_train = dataset['dataset']['train']
    data_test = dataset['dataset']['test']
    x_train = data_train[0,0]['images'][0,0]
    x_train = x_train.astype('float32') / 255
    y_train = data_train[0,0]['labels'][0,0]
    x_test = data_test[0,0]['images'][0,0]
    x_test = np.double(x_test) / 255
    y_test = data_test[0,0]['labels'][0,0]

    batchsize = self.DN['batchsize']
    iterations = int(len(x_train)/batchsize)
    batchdata = np.zeros(shape=(batchsize,784,iterations), dtype='float32')
    for i in range(iterations) :
        for j in range(batchsize):
            batchdata[j, :,i] = x_train[i+j, :]

    tensor_x = torch.Tensor(batchdata)
    tensor_y = torch.Tensor(y_train)
    x_trainGPU = torch.from_numpy(batchdata)
    return (x_train, y_train, x_test, y_test, x_trainGPU)
  
  def train_dbn(self, batchdata): 
    for layer in range(0, self.DN['nlayers']):

        print ('Training layer ', layer+1, '...')
        if layer == 0:
            data_GPU = batchdata
        else:
            data_GPU = batchposhidprobs

        numhid = self.DN['layersize'][layer]
        numcases, numdims, numbatches = data_GPU.shape
        vishid_GPU       = 0.1 * torch.randn(numdims, numhid)
        hidbiases_GPU    = torch.zeros(numhid)
        visbiases_GPU    = torch.zeros(numdims)
        vishidinc_GPU    = torch.zeros((numdims, numhid))
        hidbiasinc_GPU   = torch.zeros(numhid)
        visbiasinc_GPU   = torch.zeros(numdims)
        batchposhidprobs = torch.zeros((numcases, numhid, numbatches))


        sigmoid = nn.Sigmoid()
        for epoch in range(self.DN['maxepochs']):
            for mb in range(numbatches):
                data_mb = data_GPU[:, :, mb]

                #%%%%%%%% START POSITIVE PHASE %%%%%%%%%
                #print(data_mb.shape)
                #print(vishid_GPU.shape)
                #1./(1 + exp(-poshidstates * vishid' - repmat(visbiases, numcases, 1))); 
                poshidprobs_GPU = sigmoid((torch.mm(data_mb, vishid_GPU) + hidbiases_GPU))
                posprods_GPU    = torch.mm(data_mb.T, poshidprobs_GPU)
                poshidact_GPU   = poshidprobs_GPU.sum(0)
                posvisact_GPU   = data_GPU[:, :, mb].sum(0)
                #%%%%%%%% END OF POSITIVE PHASE %%%%%%%%%
                poshidstates_GPU = poshidprobs_GPU > poshidprobs_GPU.rand()

                #%%%%%%%% START NEGATIVE PHASE  %%%%%%%%%
                negdata_GPU     = sigmoid((torch.mm(poshidstates_GPU, vishid_GPU.T) + visbiases_GPU))
                neghidprobs_GPU = sigmoid((torch.mm(negdata_GPU, vishid_GPU) + hidbiases_GPU))
                negprods_GPU    = torch.mm(negdata_GPU.T, neghidprobs_GPU)
                neghidact_GPU   = neghidprobs_GPU.sum(0)
                negvisact_GPU   = negdata_GPU.sum(0)
                #%%%%%%%% END OF NEGATIVE PHASE %%%%%%%%%

                #%%%%%%%% UPDATE WEIGHTS AND BIASES %%%%%%%%%
                vishidinc_GPU  = vishidinc_GPU  * self.momentum + ((posprods_GPU - negprods_GPU) / numcases - self.weightcost * vishid_GPU) * epsilonw_GPU
                visbiasinc_GPU = visbiasinc_GPU * self.momentum + (posvisact_GPU - negvisact_GPU) * (self.epsilonvb / numcases)
                hidbiasinc_GPU = hidbiasinc_GPU * self.momentum + (poshidact_GPU - neghidact_GPU) * (self.epsilonhb / numcases)
                vishid_GPU     = vishid_GPU + vishidinc_GPU
                visbiases_GPU  = visbiases_GPU + visbiasinc_GPU
                hidbiases_GPU  = hidbiases_GPU + hidbiasinc_GPU
                #%%%%%%%% END OF UPDATES %%%%%%%%%

                if epoch == self.DN['maxepochs']-1:
                    batchposhidprobs[:, :, mb] = poshidprobs_GPU

                #TODO: Sparsity

        # save learned weights
        self.DN['vis_bias' + str(layer)] = visbiases_GPU.as_numpy_array()
        self.DN['hid_bias' + str(layer)] = hidbiases_GPU.as_numpy_array()
        self.DN['vishid'   + str(layer)] = vishid_GPU.as_numpy_array()

  

In [219]:
fname = ('/home/gurjeet/dbn/emnist-letters.mat')

layers_nn = np.array([50, 50, 200])

x1 = time.strftime('%s')
dbn_model = DBN(layers_nn = layers_nn)
x_train, y_train, x_test, y_test, x_trainGPU = dbn_model.load_data(fname)

x2 = time.strftime('%s')
timediff = int(x2) - int(x1)
#print '\nElapsed time: ', timediff, ' seconds.'
#DN['learningtime'] = timediff
# save final network and parameters
#scipy.io.savemat('DN.mat', {'DN': DN})

In [220]:
dbn_model.train_dbn(x_trainGPU)

Training layer  1 ...


AttributeError: ignored

In [205]:
data_mb.shape
vishid_GPU.shape

NameError: ignored